In [1]:
import logging
from typing import Any, Set

from automata.config.config_base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.context_providers.symbol_synchronization_context import (
    SymbolProviderSynchronizationContext,
)
    
from automata.tools.tool_base import Tool
from automata.experimental.tools.builders.advanced_context_oracle_builder import AdvancedContextOracleToolkitBuilder

logger = logging.getLogger(__name__)

py_module_loader.initialize()

In [2]:
symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')
symbol_doc_embedding_handler = dependency_factory.get('symbol_doc_embedding_handler')
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')

In [3]:
context_oracle = AdvancedContextOracleToolkitBuilder(symbol_search, symbol_doc_embedding_handler, symbol_code_embedding_handler, embedding_similarity_calculator)

In [4]:
context = context_oracle._get_context("Provide new code which refactors the SymbolDocEmbeddingHandler class to be more robust.")

100%|██████████| 791/791 [00:06<00:00, 118.11it/s]


In [5]:
print(context)



def _create_new_embedding(self, source_code: str, symbol: Symbol) -> None:
    'Creates a new embedding for a symbol.'
    if (symbol.py_kind == SymbolDescriptor.PyKind.Class):
        logger.debug(f'Creating a new class embedding for {symbol}')
        symbol_embedding = self.embedding_builder.build(source_code, symbol)
    elif isinstance(self.embedding_builder, SymbolDocEmbeddingBuilder):
        logger.debug(f'Creating a new non-class embedding for {symbol}')
        symbol_embedding = self.embedding_builder.build_non_class(source_code, symbol)
    else:
        raise ValueError('SymbolDocEmbeddingHandler requires a SymbolDocEmbeddingBuilder')
    self.embedding_db.add(symbol_embedding)
    logger.debug('Successfully added...')
Documentation:

Symbol: automata.experimental.memory_store.symbol_doc_embedding_handler.SymbolDocEmbeddingHandler._create_new_embedding
Creates a new embedding for a symbol.Fetching related context now for 10 symbols...

automata.symbol_embedding.symbol_em